# **Case Técnico – Cientista de Dados Júnior | Datarisk (Análise Exploratória -> EDA)**

No segundo notebook, aprofunda-se a compreensão das bases por meio de uma análise exploratória estruturada, com o objetivo de identificar padrões relevantes, comportamentos de risco e potenciais sinais de vazamento de informação.

A EDA está organizada em blocos que abordam diferentes perspectivas:


* **Construção da variável alvo:**
  A partir da regra proposta para a elaboração de um cliente inadimplente e adimplente, constroi-se o target que será avaliado durante o caso.

* **Análise univariada:**
  Explora-se a distribuição das variáveis numéricas e categóricas, avaliando assimetrias, outliers e predominância de categorias, além de mapear possíveis variáveis com baixa variabilidade.

* **Análise Multivariada:**
  São analisados indicadores proxy de atraso para identificar variáveis com impacto potencial no risco.

# 1. Importação das bibliotecas

- pandas: Manipulação dos dados
- numpy: Manipulação numérica
- eda_utils: Pacote para abstrair boiler plates
- matplotlib e seaborn: Visualização dos dados
- sklearn: Separação em treino e teste

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from src.eda_utils import *


pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
mpl.style.use('ggplot')
mpl.rcParams['axes.facecolor']      = 'white'
mpl.rcParams['grid.color']          = 'lightgray'
mpl.rcParams['xtick.color']         = 'black'
mpl.rcParams['ytick.color']         = 'black'
mpl.rcParams['axes.grid']           = True
mpl.rcParams['figure.dpi']          = 150

# Palette
instyle_palette = ['#006bbe', '#8a817c', '#254278', "#14155E", '#96c8e4']
progress_palette = ['#00215d', '#00468b', '#0071bc', '#589fef', '#8fd0ff']


sns.set_palette(sns.color_palette(instyle_palette))
sns.palplot(sns.color_palette(instyle_palette))
sns.palplot(sns.color_palette(progress_palette))


### **2. Construção do target**

Para este desafio, definiu-se que um cliente deve ser considerado inadimplente quando o pagamento ocorrer com 5 dias ou mais de atraso em relação à data de vencimento. Como essa classificação não está presente no dataset, o target deve ser construído a partir das variáveis disponíveis.

Assim, adotaremos a seguinte regra:

- TARGET = 1 → quando DATA_PAGAMENTO - DATA_VENCIMENTO ≥ 5
- TARGET = 0 → caso contrário



In [ ]:
base_pag = read_data(file='base_pagamentos_desenvolvimento.parquet', file_type='parquet', processed=True)

base_pag.head()

O primeiro passo passo para a construção é a comparação entre os dias de atraso, dessa forma, utilizaremos as colunas DATA_PAGAMENTO e DATA_VENCIMENTO para compor esta nova coluna.

In [ ]:
base_pag['DIAS_ATRASO'] = (base_pag['DATA_PAGAMENTO'] - base_pag['DATA_VENCIMENTO']).dt.days


base_pag.head()

Com os dias de atraso definidos, podemos então, construir o **target** a partir da regra proposta.

In [ ]:
base_pag['INADIMPLENTE'] = np.where(base_pag['DIAS_ATRASO'] >= 5, 1, 0)

base_pag.head()

### **3. Separação entre treino e teste (dados de validação)**

Após a construção correta da variável target, podemos avançar para a análise exploratória e posteriormente para a modelagem. No entanto, como o objetivo do projeto é prever a probabilidade de inadimplência **em períodos futuros**, é essencial que a avaliação do modelo respeite a dimensão temporal dos dados.

Por esse motivo, a separação entre dados de treino e teste será feita utilizando a estratégia Out-of-Time split (OOT).
Diferentemente de um train/test split aleatório tradicional, o OOT garante que:
- O modelo seja treinado apenas com informações do passado, e seja avaliado em um período posterior, nunca visto, reproduzindo de forma mais fiel o cenário operacional real da empresa.

Essa abordagem evita vazamento temporal (data leakage) e assegura que o desempenho medido no conjunto de teste represente a capacidade real do modelo de generalizar para meses ainda não observados.

In [ ]:
base_pag.sort_values('SAFRA_REF', inplace=True)
train, test = np.split(base_pag, [int(.80 * len(base_pag))])

train['SET'] = 'train'
test['SET'] = 'test'

In [ ]:
print(f"Taxa de inadimplente conjunto de treino:\n {train['INADIMPLENTE'].value_counts(normalize=True)}.")
print()
print(f"Taxa de inadimplente conjunto de teste:\n {test['INADIMPLENTE'].value_counts(normalize=True)}.")

Observamos taxas semelhantes de inadimplência em ambos os conjuntos, o que mostra que em termos de representativade, o split foi bem sucedido. Além disso, nota-se o desbalanceamento da variável alvo, isso será levado em conta durante a modelagem.

In [ ]:
print(f'O conjunto de treino tem {train.shape[0]} linhas e {train.shape[1]} colunas')
print(f'O conjunto de teste tem {test.shape[0]} linhas e {test.shape[1]} colunas')

In [ ]:
train_test = pd.concat([train, test])
train_test = (
    train_test
    .groupby(['SAFRA_REF', 'SET'])['INADIMPLENTE']
    .count()
    .reset_index()
)


fig, ax = plt.subplots(figsize=(12, 4))
plt.title('Out-of-time Split', fontsize=15)

for set_name, df_set in train_test.groupby('SET'):
    ax.plot(df_set['SAFRA_REF'], df_set['INADIMPLENTE'], marker='o', label=set_name)

ax.set_ylabel('Quantidade de empréstimos')
ax.legend()
plt.grid(False)
plt.tight_layout()
plt.show()


O gráfico mostra que o OOT-split é consistente e bem definido:
- O conjunto de teste representa um período futuro com volume semelhante ao do treino, preservando tendência e sazonalidade mensais. Isso evita data leakage e garante uma avaliação realista da capacidade de generalização temporal do modelo.
- Podemos observar variações relevantes no volume de empréstimos ao longo do tempo. Entre o final de 2019 e meados de 2020 há uma redução gradual nas emissões, possivelmente refletindo mudanças sazonais ou condições econômicas adversas. Um comportamento semelhante, porém mais acentuado, ocorre no final de 2020 e início de 2021, quando o volume cai de forma mais brusca antes de voltar a subir. Esses movimentos reforçam a importância do uso do Out-of-Time split, pois evidenciam variações temporais que o modelo precisa ser capaz de generalizar.

### **4. Análise univariada**

**Base de pagamentos**

In [ ]:
train[['VALOR_A_PAGAR', 'TAXA', 'DIAS_ATRASO', 'INADIMPLENTE']].describe().T

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

target_grouped = train.groupby(['INADIMPLENTE'])['INADIMPLENTE'].count().rename('count').reset_index().sort_values('count')

target_grouped['pct'] = target_grouped['count'] / target_grouped['count'].sum() * 100

bars = ax.bar(x=target_grouped['INADIMPLENTE'], height=target_grouped['pct'], color=instyle_palette)
for bar, pct in zip(bars, target_grouped['pct']):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, height, f'{pct:.1f}%', ha='center', va='bottom')
ax.set_title('A taxa de inadimplência é cerca de 7,4%', pad=25, fontweight='bold')
ax.set_xticks(ticks=target_grouped['INADIMPLENTE'], labels=['Maus credores', 'Bons credores'])
ax.yaxis.set_visible(False)
ax.grid(False)

In [ ]:
target_mean_train = (
    train
    .groupby('SAFRA_REF')['INADIMPLENTE']
    .mean()
    .reset_index(name='TARGET_MEAN')
)


fig, ax = plt.subplots(figsize=(12, 4))
plt.title('Taxa de inadimplência ao longo do tempo', fontsize=15)

ax.plot(
    target_mean_train['SAFRA_REF'],
    target_mean_train['TARGET_MEAN'],
    marker='o',
    label='TREINO'
)


ax.set_ylabel('Taxa de inadimplência')
plt.grid(False)
plt.tight_layout()
plt.show()

- Notamos um incremento nas taxas de inadimplência nos período do final de 2019 à Março de 2020.
- O pico de taxa de inadimplência ocorreu em Fevereiro de 2020, denotando quase que o dobro da média de taxa de inadimplência.

**4.1.2 Variáveis numéricas**

In [ ]:
num_feats = ['VALOR_A_PAGAR', 'DIAS_ATRASO']

univariate_analysis_plots(train, num_feats, palette=instyle_palette, histplot=True, kde=True)

- Percebemos a presença de Outliers em todas as variáveis, entretanto, os valores a serem pagos se destacam nesse quesito. Apesar de existirem cobranças de mais de 4 milhões, cerca de 75% dos valores são de até 58.000 (aproxidamente). Isso indica valores médios-altos, o que denota empresas ou contratos maiores. 
- Como já esperado, pagamentos adiantados e pontuais são a maioria, denotado pelo pico em torno de 0.


**Variáveis categóricas**

In [ ]:
grouped = (
    train['TAXA']
    .value_counts(normalize=True)
    .mul(100)
    .reset_index()
)

grouped.columns = ['TAXA', 'proportion']

grouped = grouped.sort_values('proportion')


fig, ax = plt.subplots(figsize=(6, 6))

sns.barplot(
    data=grouped,
    x='proportion',
    y='TAXA',
    palette=progress_palette,
    order=grouped['TAXA'],
    ax=ax,
    orient='h'
)
ax.set_title('5,99% é a taxa de juros mais frequente', fontweight='bold')
ax.set_xlabel('')
ax.set_ylabel('')


for bar in ax.patches:
    width = bar.get_width()
    y = bar.get_y() + bar.get_height() / 2

    ax.text(
        width + 0.5,
        y,
        f"{width:.1f}%",
        va="center",
        ha="left",
        fontsize=10
    )

ax.xaxis.set_visible(False)
ax.grid(False)
plt.show()


- Percebemos que a taxa de juros de 5,99% é a mais frequente no conjunto de treino. Ela representa aproxidamente 34% das taxas de transações de empréstimos. 
- Além disso, Quase 80% das taxas estão distríbuidas entre: 4,99%, 5,99%, 6,99%.

Como se comportam os outliers?

In [ ]:
outliers = detect_outliers_iqr(train['VALOR_A_PAGAR'])

print(f'Total: {outliers.shape[0]}')
print(f'PCT: {(outliers.shape[0] / train.shape[0]) * 100:.2f}%')

outliers

Vamos observar o comportamento dos clientes

In [ ]:
outliers_idxs = outliers.index

outliers_df = train.loc[outliers_idxs]

outliers_df.head(15)

In [ ]:
outliers_grouped = (
    outliers_df
    .groupby('ID_CLIENTE')
    .agg(
        mean_valor=('VALOR_A_PAGAR', 'mean'),
        n_trans=('VALOR_A_PAGAR', 'size')
    )
    .reset_index()
)

not_outlier = train.loc[~train.index.isin(outliers_df.index)]

not_out_matched = not_outlier.loc[
    not_outlier['ID_CLIENTE'].isin(outliers_df['ID_CLIENTE'].unique())
]

not_out_grouped = (
    not_out_matched
    .groupby('ID_CLIENTE')
    .agg(
        mean_valor=('VALOR_A_PAGAR', 'mean'),
        n_trans=('VALOR_A_PAGAR', 'size')
    )
    .reset_index()
)


print(f"Quantidade de clientes com OUTLIER: {outliers_grouped['ID_CLIENTE'].nunique()}\n")

print(f"Média de VALOR_A_PAGAR por cliente (OUTLIER): "
      f"{outliers_grouped['mean_valor'].mean():.2f}")
print(f"Média de transações por cliente (OUTLIER): "
      f"{outliers_grouped['n_trans'].mean():.2f}\n")

print(f"Média de VALOR_A_PAGAR por cliente (N/OUTLIER): "
      f"{not_out_grouped['mean_valor'].mean():.2f}")
print(f"Média de transações por cliente (N/OUTLIER): "
      f"{not_out_grouped['n_trans'].mean():.2f}")


A análise dos outliers mostrou que 233 clientes apresentam ao menos um mês classificado como extremo em RENDA_MES_ANTERIOR. Ao comparar o comportamento financeiro desses clientes, observamos que, nos meses considerados outliers, o valor médio das operações (VALOR_A_PAGAR) alcança aproximadamente R$ 176.960,85, enquanto a média de transações deste tipo é de apenas 13,85. 

Por outro lado, nos casos não-outliers desses mesmos clientes, o ticket médio cai para cerca de R$ 60.091,63, ao mesmo tempo em que o número médio de transações feitas aumenta substancialmente para 85,06. 

Essa relação inversa menos transações, porém de maior valor reforça que os outliers não representam distorções ou erros na base, mas sim períodos em que os clientes realizam operações esporádicas, porém de grande porte. Portanto, esses registros contêm informação relevante sobre o comportamento econômico dos clientes e devem ser mantidos no conjunto de treino para preservar o sinal financeiro capturado pelo modelo.

**Missings**

In [ ]:
find_na_ocurrences_by_ids(train['ID_CLIENTE'], train, ['VALOR_A_PAGAR'])

Observamos que a média de ocorrências de valores nulos por cliente é quase nula, cerca de 0,01%. Dessa forma, podemos imputar os valores históricos de cada cliente, utilizando ffill e bfill.

In [ ]:
train['VALOR_A_PAGAR'] = (
        train
        .groupby('ID_CLIENTE')['VALOR_A_PAGAR']
        .ffill()

    )

train['VALOR_A_PAGAR'] = train['VALOR_A_PAGAR'].fillna(train['VALOR_A_PAGAR'].median())


**Base cadastral**

Esta base só possui variáveis categóricas, dessa forma, torna-se desnecessário a análise de variáveis numéricas.

In [ ]:
base_cadastral = read_data(file='base_cadastral.parquet', file_type='parquet', processed=True)

In [ ]:
base_cadastral.head()

O primeiro passo é garantir que vejamos apenas os registros que estão presentes no conjunto de treinamento. Desa forma, vamos filtrar os registros pelos ids presentes neste conjunto.

In [ ]:
ids_train = train['ID_CLIENTE'].unique()

train_bc = base_cadastral[base_cadastral['ID_CLIENTE'].isin(ids_train)]

In [ ]:
cat_feats = ['FLAG_PF', 'SEGMENTO_INDUSTRIAL', 'DOMINIO_EMAIL', 'PORTE']

n = len(cat_feats)
n_cols = 2
n_rows = (n + 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 6 * n_rows))
axes = axes.flatten()

train_bc_copy = train_bc.copy()
train_bc_copy['FLAG_PF'] = train_bc_copy['FLAG_PF'].map(lambda x: 'SIM' if x == 1 else 'NÃO')

for ax, col in zip(axes, cat_feats):
    grouped = (
        train_bc_copy[col]
        .value_counts(normalize=True)
        .mul(100)
        .reset_index()
    )
    grouped.columns = [col, 'pct']
    grouped = grouped.sort_values('pct', ascending=False)

    sns.barplot(data=grouped, x=col, y='pct', ax=ax)
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticks([])

    for p in ax.patches:
        height = p.get_height()
        ax.text(
            p.get_x() + p.get_width() / 2,
            height + 0.5,
            f'{height:.1f}%',
            ha='center',
            va='bottom',
            fontsize=9
        )

    ax.set_ylim(0, grouped['pct'].max() * 1.15)
    ax.grid(False)

for j in range(len(cat_feats), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


- Percebemos a presença massiva de clientes que não são pessoa física (empresas), cerca de pouco mais de 95%.
- O segmento industrial que mais está presente no conjunto de dados é o de Serviços, representando 40% dos clientes.
- Domínios de e-mail como: GMAIL, YAHOO e HOTMAIL são os mais presentes no conjunto de dados, somados representam 86% de todos os registros.
- Cerca de 79% dos clientes estão classificados com seu porte em médio e grande. Isso reforça a presença de contratos de médio-alto valor, o qual vimos durante a análise da base de pagamentos.

**DDD e CEP**

In [ ]:
mask_valid = train_bc_copy['DDD'] != 'INVÁLIDO'

train_bc_copy['REGIAO_DDD'] = np.where(
    mask_valid,
    train_bc_copy['DDD'].str[0],
    np.nan
)

train_bc_copy['REGIAO_DDD'].head(10)

In [ ]:
cep_str = (
    train_bc_copy['CEP_2_DIG']
    .astype('Int64')
    .astype(str)
    .where(train_bc_copy['CEP_2_DIG'].notna(), other=np.nan)  # volta NaN onde era NaN
)

mask_valid = cep_str.notna()


train_bc_copy['REGIAO_CEP'] = np.where(
    mask_valid,
    cep_str.str[0],
    np.nan
)

In [ ]:
grouped_bc_ddd = (
    train_bc_copy['REGIAO_DDD']
    .value_counts(normalize=True)
    .mul(100)
    .reset_index()
    .sort_values('proportion')
)

grouped_bc_cep = (
    train_bc_copy['REGIAO_CEP']
    .value_counts(normalize=True)
    .mul(100)
    .reset_index()
    .sort_values('proportion')
)

fig, axes = plt.subplots(2, 1, figsize=(6, 10))
axes = axes.flatten()

sns.barplot(
    data=grouped_bc_ddd,
    x='REGIAO_DDD',
    y='proportion',
    palette=instyle_palette,
    ax=axes[0]
)
axes[0].set_title('Distribuição de DDD por Região', fontweight='bold')
axes[0].set_xlabel('')
axes[0].set_ylabel('')
axes[0].yaxis.set_visible(False)
axes[0].grid(False)

for bar in axes[0].patches:
    x = bar.get_x() + bar.get_width() / 2
    y = bar.get_height()
    axes[0].text(
        x=x,
        y=y,
        s=f'{y:.2f}%',
        color='black',
        va='bottom', ha='center'
    )

# CEP
sns.barplot(
    data=grouped_bc_cep,
    x='REGIAO_CEP',
    y='proportion',
    palette=instyle_palette,
    ax=axes[1]
)
axes[1].set_title('Distribuição de CEP por Região', fontweight='bold')
axes[1].set_xlabel('')
axes[1].set_ylabel('')
axes[1].yaxis.set_visible(False)
axes[1].grid(False)

for bar in axes[1].patches:
    x = bar.get_x() + bar.get_width() / 2
    y = bar.get_height()
    axes[1].text(
        x=x,
        y=y,
        s=f'{y:.2f}%',
        color='black',
        va='bottom', ha='center'
    )

plt.tight_layout()
plt.show()


- Percebemos que a região 1 (São Paulo) por DDD é mais presente no conjunto de treinamento
- Confirmando a afirmação anterior, percebemos que a região 1 (CEPs de São Paulo) são as mais presentes em registros.

**Base info**

Esta base só possui variáveis numéricas, portanto, torna-se desnecessária a análise de variáveis categóricas.

In [ ]:
base_info = read_data(file='base_info.parquet', file_type='parquet', processed=True)

base_info.head()

Assim como na 'base_cadastral', é essencial garantir que todos os registros correspondam com os IDs presentes no conjunto de treinamento de pagamentos, dessa forma, vamos novamente filtrar e garantir que essa correspondência ocorra.

In [ ]:
train_bi = base_info[base_info['ID_CLIENTE'].isin(ids_train)]

In [ ]:
num_feats = ['RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']
from matplotlib.ticker import FuncFormatter, PercentFormatter

fig, axes = plt.subplots(2, 2, figsize=(16, 8))

for row, col in enumerate(num_feats):
    s = train_bi[col].dropna()
    lo = s.quantile(0.01)
    hi = s.quantile(0.99)
    s_clip = s.clip(lo, hi)

    ax_hist = axes[row, 0]
    sns.histplot(x=s_clip, kde=True, bins=30, ax=ax_hist, palette=instyle_palette)
    ax_hist.set_title(f"{col} - Histograma")
    ax_hist.set_xlabel('')
    ax_hist.set_ylabel('Freq.')
    ax_hist.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{x:,.0f}".replace(",", ".")))
    ax_hist.grid(False)
    ax_hist.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{x:,.0f}".replace(",", ".")))

    ax_box = axes[row, 1]
    sns.boxplot(x=s, ax=ax_box, palette=instyle_palette, orient="h")
    ax_box.set_title(f"{col} - Boxplot")
    ax_box.set_xlabel('')
    ax_box.set_ylabel('')
    ax_box.grid(False)
    ax_box.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{x:,.0f}".replace(",", ".")))

plt.tight_layout()
plt.show()

- Percebemos a distorção na cauda a direita da variável de RENDA_MES_ANTERIOR. Isso denota a presença de Outliers, que podem chegar a 1.750.000. Entretanto, sua média fica por volta de 250.000,00.
- Notamos que grande parte dos clientes possuem ao menos mais de 50 funcionários. Mais uma vez reforçando o porte e tamanho dos contratos já discutidos.

Analisaremos agora a presença dos outliers

In [ ]:
outliers = detect_outliers_iqr(train_bi['RENDA_MES_ANTERIOR'])

print(f'Quantidade de outliers: {outliers.count()}')
outliers.head(15)

Quantos clientes diferentes estão classificados como outliers?

In [ ]:
outliers_df = train_bi.loc[outliers.index]

print(f'Clientes únicos classificados como outliers: {outliers_df['ID_CLIENTE'].nunique()}')
print(f'Total: {outliers_df.shape[0]}')

outliers_df.head(15)

In [ ]:
train_bi['is_outlier'] = train_bi.index.isin(outliers_df.index)


train_bi.groupby('is_outlier')[['RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']].describe().T

Por hora, manteremos os Outliers que, de certa forma, em casos de modelagem como este, são comuns de serem visto. Ao realizar a análise multivariada, observaremos melhor, como se comportam estes outliers com as demais variáveis.

Vamos verificar, agora o comportamento das colunas que apresentam missings.

In [ ]:
df_train_bi_na = train_bi.loc[train_bi['RENDA_MES_ANTERIOR'].isna()]


df_train_bi_na.head(20)

In [ ]:
df_train_bi_na['ID_CLIENTE'].nunique()

In [ ]:
print(train_bi['ID_CLIENTE'].value_counts().mean())
print(train_bi['ID_CLIENTE'].value_counts().max())
print(train_bi['ID_CLIENTE'].value_counts().min())

Vamos ver algumas métricas, como:
- Média, Máximo e Mínimo da quantidade de valores NA na coluna RENDA_MES_ANTERIOR e NO_FUNCIONARIOS por ID_CLIENTE
- Média, Máximo e Mínimo da porcentagem em relação ao total de valores NA na coluna RENDA_MES_ANTERIOR e NO_FUNCIONARIOS por ID_CLIENTE

In [ ]:
metrics_renda = find_na_ocurrences_by_ids(
    id_series=df_train_bi_na['ID_CLIENTE'],
    df=df_train_bi_na,
    features=['RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']
)

metrics_renda

Observamos que nenhum cliente apresenta uma proporção elevada de valores faltantes na variável RENDA_MES_ANTERIOR. Como a maior parte das informações está presente e distribuída ao longo do tempo, é possível preencher os valores ausentes utilizando o próprio histórico do cliente. Para isso, adotamos o método ffill (forward fill), que substitui cada valor faltante pelo último valor conhecido no tempo, garantindo consistência temporal.

In [ ]:
train_bi = train_bi.sort_values(['ID_CLIENTE', 'SAFRA_REF'])

cols_to_impute = ['NO_FUNCIONARIOS', 'RENDA_MES_ANTERIOR']

for col in cols_to_impute:
    train_bi[col] = (
        train_bi
        .groupby('ID_CLIENTE')[col]
        .ffill()
    )

    for col in cols_to_impute:
        median = train_bi[col].median()
        train_bi[col] = train_bi[col].fillna(median)

train_bi.isna().sum()

### 5. Análise Multivariada

Após a análise individual das variáveis, o próximo passo é avaliar como cada uma delas se relaciona com a variável alvo. Para isso, precisamos consolidar em um único dataset todas as informações disponíveis sobre os clientes. Assim, integraremos as bases cadastral e informacional à base de pagamentos, permitindo analisar o comportamento de inadimplência de forma completa e consistente.

In [ ]:
final_train = pd.merge(
    left=train.sort_values('SAFRA_REF'),
    right=train_bc_copy,
    how='left',
    on='ID_CLIENTE'
).merge(
    on=['ID_CLIENTE', 'SAFRA_REF'],
    how='left',
    right=train_bi
)

final_train.head()

In [ ]:
final_train.drop('is_outlier', inplace=True, axis=1)

**Existem features que estão correlacionadas umas com as outras?**

In [ ]:
mask = np.zeros_like(final_train.corr(method='pearson', numeric_only=True), dtype=bool)
mask[np.triu_indices_from(mask)] = True

corr_matrix = final_train.corr(method='pearson', numeric_only=True)

plt.figure(figsize=(25, 7))
sns.heatmap(corr_matrix,linewidths=0.25, fmt=".2f", annot=True, vmin=-1, vmax=1, mask=mask)
plt.grid(False)
plt.show()

O heatmap Pearson mostra baixas correlações lineares entre as variáveis utilizadas e o target, o que é esperado em problemas de risco, onde os efeitos são não lineares e mais bem capturados por binning ou modelos de árvore.

A única correlação elevada é entre INADIMPLENTE e DIAS_ATRASO, reflexo direto da própria forma de construção do target. As demais correlações são baixas, indicando ausência de colinearidade linear relevante entre as variáveis.

**Variáveis categóricas**

**TAXA**

In [ ]:
tax_df = woe_iv_table(
    data=final_train,
    feature='TAXA',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

tax_df

In [ ]:
plot_default_woe(
    data=final_train,
    feature='TAXA',
    target='INADIMPLENTE',
    palette=instyle_palette
)

A variável TAXA apresenta baixa capacidade de separação de risco.
As diferentes faixas de taxa mostram taxas de inadimplência muito próximas entre si, variando de apenas 6.95% a 8.68%.

O gráfico de WoE reforça essa conclusão: os valores variam entre –0.17 e +0.07, indicando separação fraca e ausência de monotonicidade.

Portanto, TAXA provavelmente tem IV baixo (<0.02) e deve ser considerada variável fraca ou até removida da modelagem, dependendo da estratégia.

**SEGMENTO INDUSTRIAL**

In [ ]:
si_df = woe_iv_table(
    data=final_train,
    feature='SEGMENTO_INDUSTRIAL',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

si_df

In [ ]:
plot_default_woe(
    data=final_train,
    feature='SEGMENTO_INDUSTRIAL',
    target='INADIMPLENTE',
    palette=instyle_palette,
    figsize=(20, 4)
)

Comércio é o segmento mais seguro, com inadimplência bem abaixo da média.

Indústria e Serviços apresentam níveis de inadimplência semelhantes e mais altos.

O WoE reforça a separação: Comércio é positivo e distanciado, indicando menor risco.

O IV é baixo, então a variável não tem grande poder preditivo isoladamente, mas ainda agrega informação útil ao modelo.

**CEP**

In [ ]:
final_train_copy = final_train.copy()

cep_df = woe_iv_table(
    data=final_train_copy,
    feature='REGIAO_CEP',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

cep_df


In [ ]:
plot_default_woe(
    data=final_train,
    feature='REGIAO_CEP',
    target='INADIMPLENTE',
    palette=instyle_palette,
    figsize=(20, 4)
)


Existe clara separação entre regiões com maior e menor inadimplência.

A região 4, 5, 6 e 7 (estados do Norte e Nordeste) deve ser tratada como ponto de atenção (alta inadimplência).

A região 8 (estados da região Sul) é extremamente segura e pode puxar o score para cima.

O IV moderado indica que essa variável contribui significativamente na explicação do risco.

**DDD**

In [ ]:
ddd =  woe_iv_table(
    data=final_train_copy,
    feature='REGIAO_DDD',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

ddd

In [ ]:
plot_default_woe(
    data=final_train,
    feature='REGIAO_DDD',
    target='INADIMPLENTE',
    palette=instyle_palette,
    figsize=(20, 4)
)

DDD é um forte discriminador de risco.

Há regiões que concentram inadimplência (ex.: DDD 0, 7, 9).

Outras são muito mais seguras (ex.: DDD 1, 3, 4 5).

O IV mostra que a variável agrega valor real na predição.

**FLAG_PF**

In [ ]:
flag_pf =  woe_iv_table(
    data=final_train_copy,
    feature='FLAG_PF',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

flag_pf

In [ ]:
plot_default_woe(
    data=final_train,
    feature='FLAG_PF',
    target='INADIMPLENTE',
    palette=instyle_palette
)

Clientes pessoa física apresentam uma taxa de inadimplência significativamente superior (21,32% vs. 7,16% em PJ).

No entanto, por representarem apenas 0,29% da base, o que se reflete em seu Information Value igual a zero.

Assim, FLAG_PF denota que PFs são um grupo naturalmente mais arriscado. Entretanto, possuem poucos casos na base.

**PORTE**

In [ ]:
porte =  woe_iv_table(
    data=final_train_copy,
    feature='PORTE',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

porte

In [ ]:
plot_default_woe(
    data=final_train,
    feature='PORTE',
    target='INADIMPLENTE',
    palette=instyle_palette
    ,figsize=(20, 6)
)

A inadimplência apresenta relação inversa com o porte da empresa. Empresas de menor porte exibem taxa de inadimplência significativamente maior (9,91%), enquanto empresas grandes demonstram comportamento substancialmente mais seguro (5,31%). A variável apresenta WoE monotônico e IV = 0,10, indicando bom poder discriminatório e relevância para o modelo de crédito.

**DOMINIO**



In [ ]:
dominio =  woe_iv_table(
    data=final_train_copy,
    feature='DOMINIO_EMAIL',
    target='INADIMPLENTE',
    bad_value=1,
    bins=None
)

dominio

In [ ]:
plot_default_woe(
    data=final_train,
    feature='DOMINIO_EMAIL',
    target='INADIMPLENTE',
    palette=instyle_palette,
    figsize=(22, 4)
)

A variável DOMINIO_EMAIL apresenta diferenças moderadas entre categorias, com domínios como HOTMAIL exibindo taxas de inadimplência superiores (9,60%) e domínios como AOL apresentando melhor desempenho (4,51%). Apesar disso, o Information Value é baixo (IV ≈ 0,06), indicando fraca capacidade discriminatória. Assim, embora exista alguma correlação com o risco, a variável não deve ser levada em conta, visto que não discrimina nenhuma informação de crédito.

### **Conclusões**

A análise exploratória permitiu consolidar os principais pontos abaixo:

- **Construção do target e recorte temporal**
  - O target foi definido como atraso **≥ 5 dias** entre DATA_PAGAMENTO e DATA_VENCIMENTO.
  - A separação em treino/validação seguiu um **OOT split temporal por SAFRA_REF**, garantindo avaliação mais realista da capacidade de generalização do modelo.
  - As taxas de inadimplência entre treino e validação são semelhantes, indicando que o split está bem calibrado.

- **Base de pagamentos**
  - A inadimplência varia ao longo do tempo, com períodos de maior risco que serão capturados por variáveis temporais na modelagem.
  - O comportamento por cliente mostra que alguns têm grande volume de transações e valores elevados, o que reforça a importância de features agregadas por histórico.

- **Base cadastral**
  - A carteira é majoritariamente composta por **pessoa jurídica**, com concentração relevante em determinadas regiões (DDD e CEP, especialmente região 1 / SP).
  - Variáveis de **porte** e **segmento industrial** apresentam relação clara com o risco: empresas maiores e segmentos mais estruturados tendem a inadimplir menos.
  - Essas variáveis cadastrais se mostram boas candidatas a features importantes de perfil.

- **Base info**
  - Há concentração relevante em poucas faixas de taxa de juros (principalmente 4,99%, 5,99% e 6,99%).
  - RENDA_MES_ANTERIOR e NO_FUNCIONARIOS apresentam caudas longas e outliers, mas coerentes com a realidade de clientes heterogêneos em porte e faturamento.
  - A proporção de valores ausentes por cliente é baixa, e a estratégia de preenchimento temporal (ffill) preserva a consistência da série no tempo.
  - Essas variáveis reforçam o poder de discriminação entre clientes mais robustos e mais frágeis financeiramente.

- **Análise multivariada e relação com o risco**
  - As correlações lineares entre variáveis numéricas são, em geral, baixas, reduzindo preocupação com multicolinearidade severa.
  - Medidas de IV/WoE indicam que **DDD, CEP, PORTE, SEGMENTO, FLAG_PF e algumas taxas** possuem boa capacidade discriminante.
  - De forma geral, o risco se mostra influenciado por três blocos principais: **características temporais, geográficas e de perfil/capacidade financeira**.
